In [1]:
# A few imports
import yt
import numpy as np
import trident
import matplotlib.pyplot as pl
from yt.units import kpc,mp
from yt.fields.particle_fields import add_volume_weighted_smoothed_field
from make_light_ray_plots import _ElectronDensity, make_light_ray_plots

In [2]:
%%time
# Import some FIRE2 data
fname = '/mnt/raid-project/murray/FIRE/FIRE_2/'+\
        'm12i_res7100_md/output/snapdir_600/snapshot_600.0.hdf5'
ds = yt.load(fname)
params = yt.load('/mnt/raid-project/murray/FIRE/FIRE_2/'+\
        'm12i_res7100_md/haloes/snap600Rpep..parameter')
ad_param = params.all_data()
# The center of the Halo was recovered in another notebook
center_Halo = ad_param["halos", "particle_position"][0]
sp = ds.sphere(center_Halo, (10, "kpc"))

yt : [INFO     ] 2020-05-22 09:20:49,155 Calculating time from 1.000e+00 to be 4.355e+17 seconds
yt : [INFO     ] 2020-05-22 09:20:49,157 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2020-05-22 09:20:49,196 Parameters: current_time              = 4.3545571088051386e+17 s
yt : [INFO     ] 2020-05-22 09:20:49,197 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2020-05-22 09:20:49,198 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-05-22 09:20:49,199 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2020-05-22 09:20:49,201 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2020-05-22 09:20:49,201 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2020-05-22 09:20:49,202 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2020-05-22 09:20:49,203 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2020-05-22 09:20:49,204 Parameters: omega_radiation           = 0.

CPU times: user 35.3 s, sys: 5.53 s, total: 40.9 s
Wall time: 40.8 s


In [3]:
%%time
ang_mom_sphere = ds.sphere(center_Halo, (2, "kpc"))
norm_perp = np.array([ 0.90088129,-0.07389156,-0.42772998])
norm_parr = yt.ortho_find(norm_perp)[1]

CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 631 µs


In [4]:
# Adds the field to the data set
ds.add_field(('PartType0', '_ElectronDensity'), function = _ElectronDensity, 
             sampling_type = 'particle', units='1/cm**3')

# smooths a particle field into a continuous field
fn = add_volume_weighted_smoothed_field(
      "PartType0", "particle_position", "particle_mass", "smoothing_length",
      "density", "_ElectronDensity",ds.field_info)

# Labels the new field
ds.field_info.alias(('gas', '_ElectronDensity'), fn[0])
ds.derived_field_list.append(('gas', '_ElectronDensity'))

In [6]:
%%time
ray_parr = make_light_ray_plots(ds, center_Halo, center_Halo,sp, norm_perp,
                                norm_parr, 'highres/ray_parr')

yt : [INFO     ] 2020-05-22 10:24:47,901 Getting segment at z = 0.0: [0.48896831 0.5163354  0.54133174] unitary to [ 6.82082245e-05  3.67948604e-06 -1.94931363e-05] unitary.
yt : [INFO     ] 2020-05-22 10:24:47,904 Getting subsegment: [0.48896831061 unitary, 0.51633540239 unitary, 0.5413317425928333 unitary] to [8.581268479224136e-05 unitary, 2.227171227153768e-05 unitary, 0.0 unitary].
/home/hlamarre/.local/lib/python3.7/site-packages/yt/units/yt_array.py:1417: RuntimeWarning: invalid value encountered in true_divide
  out=out, **kwargs)
yt : [INFO     ] 2020-05-22 10:55:08,337 Getting subsegment: [8.581268479224136e-05 unitary, 2.227171227153768e-05 unitary, 1.0 unitary] to [6.820822451016666e-05 unitary, 3.6794860398333334e-06 unitary, 0.9999805068637099 unitary].
yt : [INFO     ] 2020-05-22 11:25:55,001 Saving field data to yt dataset: [0.42890248 0.         0.9033508 ].h5.
yt : [INFO     ] 2020-05-22 11:25:55,182 Parameters: current_time              = 4.3545571088051386e+17 s
yt 

AttributeError: 'YTArray' object has no attribute 'field_parameters'